In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import numpy as np

In [56]:
# FLAGSHIPS

URL = 'https://twilight-imperium.fandom.com/wiki/Flagship'

# Fetch the webpage
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Locate Each Faction Heading
headings = soup.find_all('h3')
clean_headings = []
for heading in headings:
    if heading.find('span', class_='mw-headline'):
        clean_headings.append(heading)
headings = clean_headings

# Locate Each Heading Span
spans = []
for heading in headings:
    span = heading.find('span', class_='mw-headline')
    if span:
        spans.append(span)

# Locate Faction Name from each Span
factions = []
for span in spans:
    name = span.get_text(strip=True).strip('[]')
    factions.append(name)

# Locate Each Faction Table from each Heading
tables = []
for heading in headings:
    table = heading.find_next_sibling()
    tables.append(table)
# Currently missing The Nomad's flagship v2

# Scrape Data for Each Flagship
all_flagships = []
for table in tables:
    rows = table.find_all('tr')
    name = rows[0].get_text(strip=True)
    abilities = rows[1].get_text().lstrip('\n').rstrip('\n').split('\n')
    ability = abilities[0]

    bonus_abilities = abilities[1:]
    bonus_abilities = ' & '.join(bonus_abilities)

    cells = rows[2].find_all('td')
    row_data = [cell.get_text(strip=True) for cell in cells]
    cost = row_data[0]
    combat = row_data[1]
    move = row_data[2]
    capacity = row_data[3]

    flagship_stats = {'Name': name, 'Ability': ability, 'Bonus_Abilities': bonus_abilities, 'Cost': cost, 'Combat': combat, 'Move': move, 'Capacity': capacity}
    all_flagships.append(flagship_stats)

flagships_df = pd.DataFrame(all_flagships, index=factions)
flagships_df

,Name,Ability,Bonus_Abilities,Cost,Combat,Move,Capacity
The Arborec,Duha Menaimon,"After you activate this system, you may produc...",Sustain Damage,8,7 (x2),1,5
The Argent Flight,Quetzecoatl,Other players cannot use SPACE CANNON against ...,Sustain Damage,8,7 (x2),1,3
The Barony of Letnev,Arc Secundus,Other players' units in this system lose PLANE...,"At the start of each space combat round, repai...",8,5 (x2),1,3
The Clan of Saar,Son of Ragh,Anti-Fighter Barrage 6 (x4),Sustain Damage,8,5 (x2),1,3
The Council Keleres,Artemiris,Other players must spend 2 influence to activa...,Sustain Damage,8,7 (x2),1,6
The Embers of Muaat,The Inferno,ACTION: Spend 1 token from your strategy pool ...,Sustain Damage,8,5 (x2),1,3
The Emirates of Hacan,Wrath of Kenara,After you roll a die during space combat in th...,Sustain Damage,8,7 (x2),1,3
The Empyrean,Dynamo,After any player's unit in this system or an a...,Sustain Damage,8,5(x2),1,3
The Federation of Sol,Genesis,"At the end of the status phase, place 1 infant...",Sustain Damage,8,5 (x2),1,12
The Ghosts of Creuss,Hil Colish,This ship's system contains a delta wormhole,"During movement, this ship may move before or ...",8,5,1,3


In [57]:
# OTHER SHIPS

ships_list = ['Carrier', 'Cruiser', 'Destroyer', 'Dreadnought', 'Fighter', 'War_Sun']
ships = []

for ship in ships_list:
    URL = f'https://twilight-imperium.fandom.com/wiki/{ship}'

    # Fetch the webpage
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    tables = soup.find_all('table', class_="article-table")

    for table in tables:
        name = table.find('th').get_text(strip=True)
        table = pd.read_html(StringIO(str(table)))
        table = table[0]

        ability = table.iloc[0, 1]
        row_data = table.iloc[-2].dropna()
        cost = row_data.iloc[0]
        combat = row_data.iloc[1]
        move = row_data.iloc[2]
        try:
            capacity = row_data.iloc[3]
        except IndexError:
            capacity = np.nan

        ship = {'Name': name, 'Cost': cost, 'Combat': combat, 'Move': move, 'Capacity': capacity, 'Ability': ability}
        ships.append(ship)

ships_df = pd.DataFrame(ships)
ships_df = ships_df.drop(index=22).set_index('Name')
ships_df

,Cost,Combat,Move,Capacity,Ability
Name,,,,,
Carrier,3,9,1^,4^,NaN
Carrier II,3,9,2,6,NaN
Advanced Carrier,3,9,1^,6^,NaN
Advanced Carrier II,3,9,2,8,Sustain Damage
Cruiser,2,7^,2^,-^,NaN
Cruiser II,2,6,3,1,NaN
Saturn Engine I,2,7^,2^,1^,NaN
Saturn Engine II,2,6,3,2,Sustain Damage
Destroyer,1,9^,2,NaN,Anti-Fighter Barrage 9 (x2)


In [58]:
# GROUND FORCES

ground_forces_list = ['Infantry', 'Mechs']
ground_forces = []

for ground_force in ground_forces_list:
    URL = f'https://twilight-imperium.fandom.com/wiki/{ground_force}'

    # Fetch the webpage
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    tables = soup.find_all('table', class_="article-table")

    for table in tables:
        name = table.find('th').get_text(strip=True)
        table = pd.read_html(StringIO(str(table)))
        table = table[0]

        ability = table.iloc[0, 1]
        row_data = table.iloc[-2].dropna()
        cost = row_data.iloc[0]
        combat = row_data.iloc[1]

        ground_force = {'Name': name, 'Cost': cost, 'Combat': combat, 'Ability': ability}
        ground_forces.append(ground_force)

ground_forces_df = pd.DataFrame(ground_forces)
ground_forces_df = ground_forces_df.set_index('Name')
ground_forces_df

,Cost,Combat,Ability
Name,,,
Infantry,1 (x2),8^,NaN
Infantry II,1 (x2),7,"After this unit is destroyed, roll 1 die. If t..."
Spec Ops I,1 (x2),7^,NaN
Spec Ops II,1 (x2),6,"After this unit is destroyed, roll 1 die. If t..."
Letani Warrior I,1 (x2),8^,Production 1
Letani Warrior II,1 (x2),7,"After this unit is destroyed, roll 1 die. If t..."
Crimson Legionnaire I,1 (x2),8^,"After this unit is destroyed, gain 1 commodity..."
Crimson Legionnaire II,1 (x2),7,"After this unit is destroyed, gain 1 commodity..."
Letani Behemoth,2,6,DEPLOY: When you use MITOSIS faction ability y...


In [59]:
df = flagships_df.merge(ships_df)
df

,Name,Ability,Bonus_Abilities,Cost,Combat,Move,Capacity


In [60]:
ships = {
    "Carrier": {"Cost": 3, "Combat": 9, "Move": 1, "Capacity": 4},
    "Carrier II": {"Cost": 3, "Combat": 9, "Move": 2, "Capacity": 6},
    "Advanced Carrier"
    "Cruiser": {"Cost": 2, "Combat": 7, "Move": 2, "Capacity": 1},
    "Destroyer": {"Cost": 1, "Combat": 9, "Move": 2, "Capacity": 0},
    "Dreadnought": {"Cost": 4, "Combat": 5, "Move": 1, "Capacity": 1},
}
